<a href="https://colab.research.google.com/github/AldoF95/diplomski_rad_masters_thesis/blob/main/intent_recognition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install tensorflow_text

In [ ]:
import tensorflow as tf
import tensorflow_text as tft
import tensorflow_hub as tfh
import pandas as pd
import numpy as np

In [ ]:
data_path = "/content/drive/MyDrive/chatbot-data.xlsx"

In [ ]:
preprocessor = tfh.KerasLayer('https://tfhub.dev/google/universal-sentence-encoder-cmlm/multilingual-preprocess/2')
model = tfh.KerasLayer('https://tfhub.dev/google/LaBSE/2')

In [ ]:
data = pd.read_excel(data_path)

In [ ]:
data.columns = ['text', 'action_type', 'intent', 'g_translation']

In [ ]:
data['intent'].unique()

array(['upis_na_sveuciliste', 'studomat_problemi', 'status_studenta',
       'prijava_zavrsnog_diplomskog'], dtype=object)

In [ ]:
data['intent'] = data['intent'].astype('category')
data['intent_codes'] = data['intent'].cat.codes
data.head()

,text,action_type,intent,g_translation,intent_codes
0,je li potreban prijemni ispit za upis,static_faq,upis_na_sveuciliste,Is the admission exam for enrollment,3
1,postoji li izvanredan studij,static_faq,upis_na_sveuciliste,Is there a remarkable study,3
2,koji su troskovi upisa,static_faq,upis_na_sveuciliste,What are the cost of enrollment,3
3,gdje da uplatim troskove,static_faq,upis_na_sveuciliste,where to pay spots,3
4,gdje da uplatim skolarinu,static_faq,upis_na_sveuciliste,where to pay the skolarin,3


In [ ]:
data['intent_codes'][data['intent']=='studomat_problemi'] = 1
data[data['intent']=='studomat_problemi'].head()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,text,action_type,intent,g_translation,intent_codes
22,ne mogu pristupiti studomatu,pokreni_proces,studomat_problemi,cannot access the studomat,1
23,ispit mi nije prijavljen,dynamic_faq,studomat_problemi,The exam not reported to me,1
24,prijavio sam ispit ali ga nema na popisu,static_faq,studomat_problemi,I reported the exam but there is not on the list,1
25,prijavila sam ispit ali ga nema na popisu,static_faq,studomat_problemi,I reported the exam but there is not on the list,1
26,na studomatu mi jos nije upisana ocijena,dynamic_faq,studomat_problemi,"At the Studomat, we have not yet been enrolled",1


In [ ]:
data['intent_codes'][data['intent_codes']==3] = 2

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [ ]:
codes = data['intent_codes'].unique()

In [ ]:
validation_data = pd.DataFrame(columns=data.columns)
train_data = pd.DataFrame(columns=data.columns)
for c in codes:
  sample = data[data['intent_codes']==c]
  sample = sample.sample(frac=1)
  val = sample.sample(frac=0.2)
  train = pd.concat([sample, val]).drop_duplicates(keep=False)
  validation_data = validation_data.append(val, ignore_index=True)
  train_data = train_data.append(train, ignore_index=True)

In [ ]:
validation_data['intent_codes'].value_counts()

0    20
2    11
1    11
Name: intent_codes, dtype: int64

In [ ]:
train_data['intent_codes'].value_counts()

0    78
2    45
1    43
Name: intent_codes, dtype: int64

In [ ]:
''' data_eng = data[['g_translation', 'intent_codes']]
data_eng.columns = ['text', 'intent_codes']
data_eng.head() '''

train_data_eng = train_data[['g_translation', 'intent_codes']]
validation_data_eng = validation_data[['g_translation', 'intent_codes']]
train_data_eng.columns = ['text', 'intent_codes']
validation_data_eng.columns = ['text', 'intent_codes']

In [ ]:
''' data_split = data[['text', 'intent_codes']]
data_split.head() '''

train_data_hr = train_data[['text', 'intent_codes']]
validation_data_hr = validation_data[['text', 'intent_codes']]

In [ ]:
''' data_hr_eng = data_split.append(data_eng, ignore_index=True)
data_hr_eng.info() '''

train_data_eng_hr = train_data_eng.append(train_data_hr, ignore_index=True)
validation_data_eng_hr = validation_data_eng.append(validation_data_hr, ignore_index=True)

In [ ]:
def df_to_dataset(df, shuffle=True, batch_size=16):
  df = df.copy()
  labels = df.pop('intent_codes')
  lables_cat = tf.keras.utils.to_categorical(labels, 3)
  dataset = tf.data.Dataset.from_tensor_slices((dict(df), lables_cat))
  if shuffle:
    dataset = dataset.shuffle(buffer_size=len(df))
  dataset = dataset.batch(batch_size).prefetch(batch_size)
  return dataset

In [ ]:
train_df_eng = df_to_dataset(train_data_eng)
validation_df_eng = df_to_dataset(validation_data_eng)

train_df_hr = df_to_dataset(train_data_hr)
validation_df_hr = df_to_dataset(validation_data_hr)

train_df_eng_hr = df_to_dataset(train_data_eng_hr)
validation_df_eng_hr = df_to_dataset(validation_data_eng_hr)

In [ ]:
train_df = df_to_dataset(data_split)

In [ ]:
[(text, lbl)] = train_df.take(1)
lbl

<tf.Tensor: shape=(16, 3), dtype=float32, numpy=
array([[1., 0., 0.],
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 0., 1.],
       [1., 0., 0.],
       [0., 1., 0.],
       [1., 0., 0.],
       [0., 1., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [0., 0., 1.],
       [1., 0., 0.],
       [1., 0., 0.],
       [0., 1., 0.],
       [1., 0., 0.]], dtype=float32)>

In [ ]:
train_hr_eng = df_to_dataset(data_hr_eng)

In [ ]:
LR_DECAY = tf.keras.optimizers.schedules.ExponentialDecay(
    0.001, 52, 0.96, True
)

In [ ]:
def model_build():
  inputs = tf.keras.layers.Input(shape=(), dtype=tf.string, name='text')
  encoded_input = preprocessor(inputs)
  encoder_outputs = model(encoded_input)
  x = encoder_outputs['pooled_output']
  x = tf.keras.layers.Dropout(0.1)(x)
  x = tf.keras.layers.Dense(128, activation='relu')(x)
  x = tf.keras.layers.Dropout(0.7)(x)
  outputs = tf.keras.layers.Dense(3, activation='softmax', name='classifier')(x)
  return tf.keras.Model(inputs, outputs)
  

In [ ]:
classifier_eng = model_build()
classifier_eng.compile(
    optimizer = tf.keras.optimizers.Adam(0.001),
    loss = tf.keras.losses.CategoricalCrossentropy(from_logits=True),
    metrics = tf.keras.metrics.CategoricalAccuracy()
)

history_eng = classifier_eng.fit(
    train_df_eng,
    epochs = 5,
    batch_size = 16,
    validation_data = validation_df_eng
)

Epoch 1/5


/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/backend.py:4870: UserWarning: "`categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  '"`categorical_crossentropy` received `from_logits=True`, but '


11/11 [==============================] - 109s 9s/step - loss: 1.1997 - categorical_accuracy: 0.4337 - val_loss: 0.5708 - val_categorical_accuracy: 0.8810
Epoch 2/5
11/11 [==============================] - 96s 9s/step - loss: 0.6779 - categorical_accuracy: 0.6988 - val_loss: 0.3471 - val_categorical_accuracy: 0.9762
Epoch 3/5
11/11 [==============================] - 96s 9s/step - loss: 0.3527 - categorical_accuracy: 0.8614 - val_loss: 0.2230 - val_categorical_accuracy: 0.9762
Epoch 4/5
11/11 [==============================] - 95s 9s/step - loss: 0.2811 - categorical_accuracy: 0.8976 - val_loss: 0.1713 - val_categorical_accuracy: 0.9762
Epoch 5/5
11/11 [==============================] - 96s 9s/step - loss: 0.2674 - categorical_accuracy: 0.8795 - val_loss: 0.1615 - val_categorical_accuracy: 0.9762


In [ ]:
classifier_hr = model_build()
classifier_hr.compile(
    optimizer = tf.keras.optimizers.Adam(0.001),
    loss = tf.keras.losses.CategoricalCrossentropy(from_logits=True),
    metrics = tf.keras.metrics.CategoricalAccuracy()
)

history_hr = classifier_hr.fit(
    train_df_hr,
    epochs = 5,
    batch_size = 16,
    validation_data = validation_df_hr
)

Epoch 1/5


/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/backend.py:4870: UserWarning: "`categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  '"`categorical_crossentropy` received `from_logits=True`, but '


11/11 [==============================] - 101s 9s/step - loss: 1.0563 - categorical_accuracy: 0.5181 - val_loss: 0.5690 - val_categorical_accuracy: 0.8810
Epoch 2/5
11/11 [==============================] - 97s 9s/step - loss: 0.6875 - categorical_accuracy: 0.7410 - val_loss: 0.3496 - val_categorical_accuracy: 0.9286
Epoch 3/5
11/11 [==============================] - 95s 9s/step - loss: 0.4457 - categorical_accuracy: 0.8253 - val_loss: 0.2472 - val_categorical_accuracy: 0.9524
Epoch 4/5
11/11 [==============================] - 96s 9s/step - loss: 0.3008 - categorical_accuracy: 0.9157 - val_loss: 0.1934 - val_categorical_accuracy: 0.9762
Epoch 5/5
11/11 [==============================] - 96s 9s/step - loss: 0.2505 - categorical_accuracy: 0.9096 - val_loss: 0.1630 - val_categorical_accuracy: 0.9762


In [ ]:
classifier_hr_eng = model_build()
classifier_hr_eng.compile(
    optimizer = tf.keras.optimizers.Adam(LR_DECAY),
    loss = tf.keras.losses.CategoricalCrossentropy(from_logits=True),
    metrics = tf.keras.metrics.CategoricalAccuracy()
)

history_hr_eng = classifier_hr_eng.fit(
    train_df_eng_hr,
    epochs = 10,
    batch_size = 16,
    validation_data = validation_df_eng_hr
)

Epoch 1/10


/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/backend.py:4870: UserWarning: "`categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  '"`categorical_crossentropy` received `from_logits=True`, but '


21/21 [==============================] - 196s 9s/step - loss: 0.9671 - categorical_accuracy: 0.5843 - val_loss: 0.3597 - val_categorical_accuracy: 0.9286
Epoch 2/10
21/21 [==============================] - 191s 9s/step - loss: 0.4001 - categorical_accuracy: 0.8645 - val_loss: 0.2212 - val_categorical_accuracy: 0.9762
Epoch 3/10
21/21 [==============================] - 192s 9s/step - loss: 0.2943 - categorical_accuracy: 0.8946 - val_loss: 0.1502 - val_categorical_accuracy: 0.9762
Epoch 4/10
21/21 [==============================] - 190s 9s/step - loss: 0.2181 - categorical_accuracy: 0.9217 - val_loss: 0.1266 - val_categorical_accuracy: 0.9643
Epoch 5/10
21/21 [==============================] - 191s 9s/step - loss: 0.1668 - categorical_accuracy: 0.9428 - val_loss: 0.1087 - val_categorical_accuracy: 0.9762
Epoch 6/10
21/21 [==============================] - 191s 9s/step - loss: 0.1428 - categorical_accuracy: 0.9608 - val_loss: 0.1356 - val_categorical_accuracy: 0.9762
Epoch 7/10
21/21 [===

In [ ]:
test_data = "hello"

In [ ]:
test_data = tf.expand_dims(test_data, axis=0)
prediction = classifier_hr_eng.predict(test_data)

In [ ]:
prediction2 = list(map('{:.3f}'.format,prediction[0]))
prediction2

['0.620', '0.303', '0.077']

In [ ]:
data['intent_codes'].value_counts()

0    98
2    56
1    54
Name: intent_codes, dtype: int64